In [1]:
import networkExpansionPy.lib as ne
from scipy.sparse import csr_matrix
import numpy as np
from random import sample
import pandas as pd

In [2]:
metabolism = ne.GlobalMetabolicNetwork()
metabolism.pruneUnbalancedReactions()
#metabolism.pruneInconsistentReactions()
metabolism.set_ph(7.0)
metabolism.convertToIrreversible()
metabolism.setMetaboliteBounds(ub=1e-1,lb=1e-6)
metabolism.pruneThermodynamicallyInfeasibleReactions(keepnan=False)

# build base model


In [3]:
# define seed compounds
seed = ['C00001','C00011','C00080','C00014','C00009','C00283','C00048','C00022']

In [4]:
rules = pd.read_csv('../networkExpansionPy/assets/ecode/ecodeRules.KeggReactions.csv')

In [12]:
95*2*3

570

In [5]:
# decompose rules to make matrix with rules x fold
ecode_rules = rules['e_code_rule'].apply(lambda x: [int(x) for x in x.split('_')])
ecode_folds = list(set([item for sublist in ecode_rules for item in sublist]))
folds = np.sort(list(set(ecode_folds)))

In [6]:
dfs = []
for i in range(0,len(ecode_rules)):
    dfs.append(pd.DataFrame({'rule':i,'fold': ecode_rules[i]}))
dfs = pd.concat(dfs,axis=0)

In [7]:
df_rulefold = dfs.set_index('rule').join(rules[['e_code_rule','rn']])

In [9]:
len(df_rulefold.fold.unique())

876

In [10]:
df_rulefold

,fold,e_code_rule,rn
0,7510,7510,R07105
1,7510,7510,R00623
2,7510,7510,R00754
3,7510,7510,R02124
4,7510,7510,R02878
5,7510,7510,R04805
6,7510,7510,R04880
7,7510,7510,R05233
8,7510,7510,R05234
9,7510,7510,R06917


In [56]:
len(metabolism.network[metabolism.network.cid.isin(seed)].rn.unique())

2556

In [57]:
# construct params for network expansion
network = metabolism.network.pivot_table(index='cid',columns = ['rn','direction'],values='s').fillna(0)
S = network.values
R = (S < 0)*1
P = (S > 0)*1
b = sum(R)

# sparsefy data
R = csr_matrix(R)
P = csr_matrix(P)
b = csr_matrix(b)
b = b.transpose()

In [58]:
x0 = np.array([x in seed for x in network.index.get_level_values(0)]) * 1
x0 = csr_matrix(x0)
x0 = x0.transpose()

In [59]:
y = (np.dot(R.transpose(),x0) == b);
y = y.astype('int');
nn = network.iloc[:,y.indices]
reactions = [x[0] for x in list(nn)]

/Users/joshuagoldford/miniconda2/envs/network_expansion/lib/python3.6/site-packages/scipy/sparse/compressed.py:226: SparseEfficiencyWarning: Comparing sparse matrices using == is inefficient, try using != instead.
  " != instead.", SparseEfficiencyWarning)


In [63]:
fc = df_rulefold[df_rulefold.rn.isin(reactions)].groupby('fold').count()['rn'].sort_values(ascending=False)

In [66]:
q = fc.idxmax()

In [109]:
fold_set = [];
fold_set = fold_set + [q]

In [108]:
findRules = lambda fold_set: np.where([set(x).issubset(fold_set) for x in ecode_rules])[0]

In [115]:
rn_on = rules.iloc[findRules(fold_set)]['rn'].unique().tolist()

In [126]:

subnetwork = network.loc[:,rn_on]

S = subnetwork.values
R = (S < 0)*1
P = (S > 0)*1
b = sum(R)

# sparsefy data
R = csr_matrix(R)
P = csr_matrix(P)
b = csr_matrix(b)
b = b.transpose()

x0 = np.array([x in seed for x in network.index.get_level_values(0)]) * 1
x0 = csr_matrix(x0)
x0 = x0.transpose()

y = (np.dot(R.transpose(),x0) == b);
y = y.astype('int');
x_n = np.dot(P,y) + x0;
x_n = x_n.astype('bool');
x = x_n.astype('int');

nn = subnetwork.iloc[:,y.indices]
reactions = [x[0] for x in list(nn)]
fc = df_rulefold[df_rulefold.rn.isin(reactions)].groupby('fold').count()['rn'].sort_values(ascending=False)


In [128]:
nn = subnetwork.iloc[:,y.indices]
reactions = [x[0] for x in list(nn)]
fc = df_rulefold[df_rulefold.rn.isin(reactions)].groupby('fold').count()['rn'].sort_values(ascending=False)
# oonly keep reamining folds:
fc = fc.loc[[x for x in fc.index.get_level_values(0) if x not in fold_set]]
fold_set = fold_set + [fc.idxmax()]

876